In [6]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from tqdm import tqdm_notebook as tqdm
import re
import math
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'IPAexGothic'

from scipy.cluster.hierarchy import linkage, dendrogram

from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel, CoherenceModel, TfidfModel

import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

from wordcloud import WordCloud

In [7]:
import glob
from bs4 import BeautifulSoup

files = glob.glob('./work/raw/*html')

def parse(fileName):
    with open(fileName) as f:
        soup = BeautifulSoup(f, 'html.parser')

    title      = soup.select_one('article.blog-entry-article h1.blog-title').get_text()
    date       = soup.select_one('article.blog-entry-article div.blog-date').get_text()
    category   = soup.select_one('article.blog-entry-article li.blog-category').get_text()
    text       = soup.select_one('article.blog-entry-article div.content').get_text()
    
    
    return [fileName, title, date, category, text]


data = [parse(fileName) for fileName in tqdm(files)]

  0%|          | 0/68 [00:00<?, ?it/s]

In [8]:
df = pd.DataFrame(data, columns=['file', 'title', 'date', 'category', 'text'])
df.head()

,file,title,date,category,text
0,./work/raw/23.html,ソフトウェアエンジニアの採用にルーブリックを導入した話,2019.12.22,TECH,\n\n\nソフトウェアエンジニアの hota です。今回はソフトウェアエンジニアの採用につ...
1,./work/raw/35.html,Kubernetes + Fluentd + CloudWatch Logs,2019.12.8,TECH,ソフトウェアエンジニアのskirinoです。\n最近ではコンテナ化したアプリケーションの設定...
2,./work/raw/62.html,event timeとprocessing timeについて,2018.12.4,TECH,\nこんにちは。ソフトウェアエンジニアの田中伸弥です。\n\n\n時系列データのevent ...
3,./work/raw/9.html,フライウィール・データプラットフォームの紹介,2020.8.27,MARKETING,プロダクトマネージャーの横井啓介です。\n前回の投稿では、デジタルトランスフォーメーション戦...
4,./work/raw/19.html,渋谷オフィスへの引越し前に認証を引っ越した話 Part2-実践編,2019.12.25,TECH,こんにちは。FLYWHEELでソフトウェアエンジニアをしてますsaoiです。前回の投稿の投稿...


In [32]:
df.groupby('category').agg(['count'])

,file,title,date,text,words
,count,count,count,count,count
category,,,,,
COMPANY,6,6,6,6,6
MARKETING,10,10,10,10,10
TECH,52,52,52,52,52


In [10]:
## 前処理
import MeCab
m = MeCab.Tagger('-d /usr/local/lib/mecab/dic/ipadic')

HINSHI = ['名詞']
STOPWORDS = ['フライウィール', 'flywheel', 'var', 'main', 'test', 'time', 'src', 'com', 'jp', 'れる', 'これ', 'なっ', 'それ', 'もの', 'たち', 'さん']

def parseText(text):
    node = m.parseToNode(text)
    words = []
    while node:
        fields = node.feature.split(",")
        word = node.surface.lower() # 小文字化
        word = re.sub(r'\d+', '0', word) # 数字置き換え        
        word = re.sub(r'[\.\/\(\){}\[\]:,?!;\*=_\-\'"@<>#\^%]+', '', word) # 記号除去
        if fields[0] in HINSHI and word not in STOPWORDS and len(word) > 1:
            words.append(word)
        node = node.next
    
    return words

In [11]:
df['words'] = df['text'].map(lambda text: parseText(text))
# df['words'] = df['title'].map(lambda text: parseText(text))
df['words'].head()

0    [ソフトウェア, エンジニア, hota, 今回, ソフトウェア, エンジニア, 採用, お...
1    [ソフトウェア, エンジニア, skirino, 最近, コンテナ, アプリケーション, 設...
2    [ソフトウェア, エンジニア, 田中, 系列, データ, event, processing...
3    [プロダクト, マネージャー, 横井, 啓介, 前回, 投稿, デジタルトランスフォーメーシ...
4    [ソフトウェア, エンジニア, saoi, 前回, 投稿, 投稿, 広告, 配信, プラット...
Name: words, dtype: object

In [12]:
## 辞書とコーパスの作成
dictionary = Dictionary(df['words'])
dictionary.filter_extremes(no_below=3, no_above=0.7)
print(len(dictionary))

# BoWコーパス
corpus = [dictionary.doc2bow(words) for words in df['words']]

# tfidfコーパス
tfidf = TfidfModel(corpus)
corpus = tfidf[corpus]

1388


In [13]:
## トピック数の探索
start = 2
limit = 10
step = 1

coherence_vals = []
perplexity_vals = []

for n_topic in tqdm(range(start, limit, step)):
    lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=n_topic, random_state=0)
    perplexity_vals.append(np.exp2(-lda_model.log_perplexity(corpus)))
    coherence_model_lda = CoherenceModel(model=lda_model, texts=df['words'], dictionary=dictionary, coherence='c_v')
    coherence_vals.append(coherence_model_lda.get_coherence())

  0%|          | 0/8 [00:00<?, ?it/s]

In [14]:
# evaluation
x = range(start, limit, step)

fig, ax1 = plt.subplots(figsize=(12,5))

# coherence
c1 = 'darkturquoise'
ax1.plot(x, coherence_vals, 'o-', color=c1)
ax1.set_xlabel('Num Topics')
ax1.set_ylabel('Coherence', color=c1); ax1.tick_params('y', colors=c1)

# perplexity
c2 = 'slategray'
ax2 = ax1.twinx()
ax2.plot(x, perplexity_vals, 'o-', color=c2)
ax2.set_ylabel('Perplexity', color=c2); ax2.tick_params('y', colors=c2)

# Vis
ax1.set_xticks(x)
fig.tight_layout()
plt.show()
plt.savefig('work/metrics.png')


In [15]:
NUM_TOPICS = 4
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=NUM_TOPICS, random_state=0)
lda_model.save('work/lda.model')

In [28]:
for i in range(lda_model.num_topics):
        print('TOPIC:', i, '__', lda_model.print_topic(i))

TOPIC: 0 __ 0.003*"event" + 0.003*"インターン" + 0.002*"processing" + 0.002*"ログ" + 0.002*"task" + 0.002*"クリック" + 0.002*"絵文字" + 0.002*"リポジトリ" + 0.002*"回転" + 0.002*"user"
TOPIC: 1 __ 0.003*"java" + 0.003*"bazel" + 0.002*"info" + 0.002*"アイデア" + 0.002*"お客様" + 0.002*"失敗" + 0.002*"ログ" + 0.002*"面接" + 0.002*"ユーザー" + 0.002*"cloud"
TOPIC: 2 __ 0.003*"入札" + 0.002*"インターン" + 0.002*"広告" + 0.002*"dsp" + 0.002*"テスト" + 0.002*"商品" + 0.002*"パーソナライゼーション" + 0.002*"技術" + 0.002*"file" + 0.002*"企業"
TOPIC: 3 __ 0.002*"回転" + 0.002*"auth" + 0.002*"施策" + 0.002*"企業" + 0.002*"組織" + 0.002*"dx" + 0.002*"emoji" + 0.002*"ログ" + 0.002*"認証" + 0.002*"移行"


In [17]:
# WordCloud
# 日本語フォントをダウンロードしてwork以下に設置
fig, axs = plt.subplots(ncols=2, nrows=math.ceil(lda_model.num_topics/2), figsize=(16,20))
axs = axs.flatten()

def color_func(word, font_size, position, orientation, random_state, font_path):
    return 'darkturquoise'

for i, t in enumerate(range(lda_model.num_topics)):

    x = dict(lda_model.show_topic(t, 30))
    im = WordCloud(
        background_color='black',
        color_func=color_func,
        max_words=4000,
        width=300, height=300,
        random_state=0,
        font_path='./work/ipaexg.ttf'
    ).generate_from_frequencies(x)
    axs[i].imshow(im.recolor(colormap= 'Paired_r' , random_state=244), alpha=0.98)
    axs[i].axis('off')
    axs[i].set_title('Topic '+str(t))

# vis
plt.tight_layout()
plt.show()

# save as png
plt.savefig('work/wordcloud.png') 

In [18]:
# Vis PCoA
vis_pcoa = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary, sort_topics=False)
vis_pcoa

# save as html
pyLDAvis.save_html(vis_pcoa, 'work/pyldavis_output_pcoa.html')

In [34]:
data = []
for c, words, fileName, title, category in zip(corpus, df['words'], df['file'], df['title'], df['category']):
    topics = []
    for topic, score in lda_model[c]:
        if (score > 0.7):
            topics.append(str(topic))
    data.append([fileName, title, category, ','.join(topics)])

df_topic = pd.DataFrame(data, columns=['file', 'title', 'category', 'topics'])
df_topic.head()


,file,title,category,topics
0,./work/raw/23.html,ソフトウェアエンジニアの採用にルーブリックを導入した話,TECH,1
1,./work/raw/35.html,Kubernetes + Fluentd + CloudWatch Logs,TECH,0
2,./work/raw/62.html,event timeとprocessing timeについて,TECH,0
3,./work/raw/9.html,フライウィール・データプラットフォームの紹介,MARKETING,1
4,./work/raw/19.html,渋谷オフィスへの引越し前に認証を引っ越した話 Part2-実践編,TECH,3


In [35]:
for i in range(lda_model.num_topics):
        print('## TOPIC:', i)
        print('\n'.join(df_topic[df_topic['topics'].str.contains(str(i))]['category']))
        print()

## TOPIC: 0
TECH
TECH
COMPANY
TECH
TECH
TECH
MARKETING
TECH
COMPANY
TECH
TECH
TECH
TECH
TECH
TECH
TECH
COMPANY
TECH
TECH
TECH
TECH
TECH
TECH
TECH
TECH
TECH

## TOPIC: 1
TECH
MARKETING
TECH
TECH
TECH
TECH
TECH
MARKETING
TECH
MARKETING
TECH
TECH
TECH
TECH
TECH
TECH

## TOPIC: 2
COMPANY
TECH
MARKETING
TECH
TECH
TECH
TECH
TECH
TECH
TECH
TECH
MARKETING

## TOPIC: 3
TECH
TECH
MARKETING
TECH
TECH
TECH
COMPANY
TECH
MARKETING
TECH
TECH
MARKETING
MARKETING
COMPANY



,file,title,topics
4,./work/raw/19.html,渋谷オフィスへの引越し前に認証を引っ越した話 Part2-実践編,3
11,./work/raw/55.html,FLYWHEEL アドベントカレンダーを支える技術,3
14,./work/raw/4.html,近未来のマーケティングを実現するパーソナライゼーション エンジンとは？,3
16,./work/raw/59.html,FLYWHEEL って、なに？,3
28,./work/raw/25.html,社内Slackの絵文字事情を調査する Part 3. 統計分析編,3
34,./work/raw/32.html,渋谷オフィスへの引越し前に認証を引っ越した話 Part1-導入編,3
38,./work/raw/12.html,急成長やリモートワーク化を支えるコミュニケーション設計,3
41,./work/raw/50.html,BtoB Startup における Product Manager,3
43,./work/raw/11.html,顧客体験のパーソナライズに必要な３つの仕掛け,3
49,./work/raw/30.html,とあるエンジニアがWFHしたときのとっても詳細な1日の流れ,3
